In [156]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [157]:
df = pd.read_csv("C:\\Users\\rohit\\OneDrive\\Desktop\\Data Analyst\\PortfolioProjects\\LLM Project\\sample_text.csv")
df.shape

(8, 2)

In [158]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [159]:
# Step 1 : Create source embeddings for the text column

In [160]:
from sentence_transformers import SentenceTransformer

In [161]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

C:\Users\rohit\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [162]:
vectors.shape

(8, 768)

In [163]:
dim = vectors.shape[1]
dim

768

In [164]:
# Step 2 : Build a FAISS Index for vectors

In [165]:
import faiss

index = faiss.IndexFlatL2(dim)

In [166]:
# Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [167]:
index.add(vectors)

In [168]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000142BA20A9D0> >

In [169]:
# Step 4 : Encode search text using same encorder and normalize the output vector

In [170]:
search_query = "Where should i go next"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [171]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [172]:
faiss.normalize_L2(svec)

In [173]:
# Step 5: Search for similar vector in the FAISS index created

In [174]:
distances, I = index.search(svec, k=2)
I

array([[6, 7]], dtype=int64)

In [175]:
df.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
